In [12]:
%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:

master_data_filepath = '..\\2024-12-04 Data summary v3 - All geos.xlsx'
transit_filepath = '..\\CSDs_TransitStatsFormattedDataV2.csv'

db_path = '..\\sources\\hicc.db'


In [14]:
# The following code uploads data to database
from DBUploader import DBUploader
uploader = DBUploader(master_data_filepath, transit_filepath, db_path)



Reading Input master data...
Input master data loaded...
Preparing Output 1
Output 1 Successfully created...
Preparing Output 1
Output 1 Successfully created...
Preparing Output 2...
Output 2 Successfully created...
Preparing Output 3...
Output 3 Successfully created...
Preparing Output 4a...
Output 4a Successfully created...
Preparing Output 4b...
Output 4b Successfully created...
Preparing Output 5a...
Output 5a Successfully created...
Preparing Output 5b...
Output 5b Successfully created...
Preparing Output 6...
Output 6 Successfully created...
Preparing Output 7...
Output 7 Successfully created...
Preparing Output 8...
Output 8 Successfully created...
Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Output 10 a and b Successfully created...
Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Output 10 a and b Successfully created...
Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Output 10 a and b 

In [15]:
uploader()

Database ready....


### Test

In [3]:
from PrepareTables import PrepareTables
pt = PrepareTables(master_data_filepath, transit_filepath)
pt.prepare_output_1()

Reading Input master data...
Input master data loaded...
Preparing Output 1
Some columns from output 1 were not found


UnboundLocalError: local variable 'output_1' referenced before assignment

In [29]:
# output_5a = p.prepare_output_5a()

# p.prepare_output_5b(output_5a)

p.prepare_output_9_and_10()

Preparing Output 9...
Output 9 Successfully created...
Preparing Output 10...
Output 10 a and b Successfully created...


(                                                                    2016 Population  \
 Geography       GEO_TYPE_ABBR_EN ALT_GEO_CODE_EN PR_CODE_EN Age                       
 Abbotsford      CY               5909052         59         15to24          18260.0   
                                                             25to34          19110.0   
                                                             35to44          17825.0   
                                                             45to54          18500.0   
                                                             55to64          17860.0   
 ...                                                                             ...   
 Île-à-la-Crosse NV               4718067         47         55to64            130.0   
                                                             65to74             70.0   
                                                             75plus              NaN   
                                

In [3]:
def geo_code_extractor(geography):
    geo = geography.split()
    # print(geo)
    for g in geo:
        if g[0] == '(' and g[1].isdigit():
            g = g.replace("(", "")
            g = g.replace(")", "")
            break
    return g

In [4]:
geo_code_list = geo_code.apply(lambda x: geo_code_extractor(x))
region_code_list = geo_code_list.apply(lambda x: x[:4])
province_code_list = geo_code_list.apply(lambda x: x[:2])

In [8]:
geo_code_mapping = pd.DataFrame({'Geo_Code': geo_code_list, 'Region_Code': region_code_list, 'Province_Code': province_code_list, 'Geography': df_income['Formatted Name']})
geo_code_mapping['Geo_Code_Length'] = geo_code_mapping['Geo_Code'].apply(lambda x: len(x))

region_code_mapping = geo_code_mapping.loc[geo_code_mapping['Geo_Code_Length'] == 4, :]
province_code_mapping = geo_code_mapping.loc[geo_code_mapping['Geo_Code_Length'] <= 2, :]

mapped_geo_code = geo_code_mapping.merge(region_code_mapping[['Geo_Code','Geography']], how = 'left', left_on = 'Region_Code', right_on = 'Geo_Code')
mapped_geo_code = mapped_geo_code.merge(province_code_mapping[['Geo_Code','Geography']], how = 'left', left_on = 'Province_Code', right_on = 'Geo_Code')
mapped_geo_code = mapped_geo_code[['Geo_Code_x', 'Region_Code', 'Province_Code', 'Geography_x','Geography_y','Geography']]
mapped_geo_code.columns = ['Geo_Code', 'Region_Code', 'Province_Code', 'Geography','Region','Province']
# mapped_geo_code['Region'] = mapped_geo_code['Region'].fillna(mapped_geo_code['Province'])

In [23]:
mapped_geo_code[mapped_geo_code['Geography']=='Vancouver CY (CSD, BC)']

,Geo_Code,Region_Code,Province_Code,Geography,Region,Province
809,5915022,5915,59,"Vancouver CY (CSD, BC)","Greater Vancouver (CD, BC)",British Columbia (Province)


In [14]:
from sqlalchemy import create_engine, inspect
import os
import pandas as pd
# current_dir = os.path.dirname(os.path.abspath(__file__))
engine_new = create_engine('sqlite:///../sources//hicc.db')


In [15]:
output_1a = pd.read_sql_table('Output_1a', engine_new.connect())
output_1a
# inspector = inspect(engine_new)
# tables = inspector.get_table_names()
# tables

,pk
